In [1]:
import os
from glass.geo.gt.torst import rsts_to_gpkg

gpkg = {
    "dstfire_clc18_10x.gpkg"  : '/mnt/d/gwork/geodb_gpkg/rstfire_clc18',
    "dstfire_cos18_10x.gpkg"  : '/mnt/d/gwork/geodb_gpkg/rstfire_cos18',
    "dstfire_fty15_10x.gpkg"  : '/mnt/d/gwork/geodb_gpkg/rstfire_fty15',
    "dstfire_ua12_10x.gpkg"   : '/mnt/d/gwork/geodb_gpkg/rstfire_ua12',
    "dstflood_clc18_10x.gpkg" : '/mnt/d/gwork/geodb_gpkg/rstflood_clc18',
    "dstflood_cos18_10x.gpkg" : '/mnt/d/gwork/geodb_gpkg/rstflood_cos18',
    "dstflood_ua12_10x.gpkg"  : '/mnt/d/gwork/geodb_gpkg/rstflood_ua12',
    "dstflood_waw15_10x.gpkg" : '/mnt/d/gwork/geodb_gpkg/rstflood_waw15',
    "dsttrans_clc18_10x.gpkg" : '/mnt/d/gwork/geodb_gpkg/rsttrans_clc18',
    "dsttrans_cos18_10x.gpkg" : '/mnt/d/gwork/geodb_gpkg/rsttrans_cos18',
    "dsttrans_ua12_10x.gpkg"  : '/mnt/d/gwork/geodb_gpkg/rsttrans_ua12',
    "dsturban_clc18_10x.gpkg" : '/mnt/d/gwork/geodb_gpkg/rsturban_clc18',
    "dsturban_cos18_10x.gpkg" : "/mnt/d/gwork/geodb_gpkg/rsturban_cos18",
    "dsturban_imd15_10x.gpkg" : "/mnt/d/gwork/geodb_gpkg/rsturban_imd15",
    "dsturban_ua12_10x.gpkg"  : "/mnt/d/gwork/geodb_gpkg/rsturban_ua12"
}

out_folder = '/mnt/d/gwork/geodb_gpkg'
new_name = 'cell_' 

for g in gpkg:
    rsts_to_gpkg(gpkg[g], os.path.join(out_folder, g), rst_ff='.tif', basename=new_name)

In [ ]:
import os
from glass.pyt.oss import lst_ff
from glass.geo.gt.torst import conv_rst_dtype

rst_folder = '/mnt/f/gwork/fireloc_db/dist_lulc_cls/rsturban_imd15'
out_folder = '/mnt/d/gwork/geodb_gpkg/rsturban_imd15'
otype      = 'uint16'

rsts = lst_ff(rst_folder, file_format='.tif')

for rst in rsts:
    conv_rst_dtype(rst, os.path.join(out_folder, os.path.basename(rst)), otype)

### Convert All Rasters in Folder to Shapefile ###

In [ ]:
# Convert using GRASS GIS

import os
from gasp.to.shp  import rst_to_polyg
from gasp.oss     import lst_ff
from gasp.cpu.grs import run_grass
from gasp.oss import fprop

GRASS_WORKSPACE = r'D:\indev\xyz'
GRASS_LOCATION  = 'conv_viena'
RST_FOLDER = r'D:\indev\OSM2LULC\WEB_Via\results_fev19\viena'
OUT_FOLDER = r'D:\indev\OSM2LULC\WEB_Via\results_fev19\viena'
SRS = 3857
RST_TEMPLATE = r'D:\indev\OSM2LULC\WEB_Via\results_fev19\viena\v14_viena_2x2.tif'

# List Raster Files
rstFiles = lst_ff(RST_FOLDER, file_format='tif')

# Start GRASS GIS Session
grsbase = run_grass(
    GRASS_WORKSPACE, location=GRASS_LOCATION, srs=SRS)

import grass.script.setup as gsetup
gsetup.init(grsbase, GRASS_WORKSPACE, GRASS_LOCATION, 'PERMANENT')

from gasp.to.rst.grs import rst_to_grs
from gasp.cpu.grs.conf import rst_to_region
from gasp.to.shp.grs import grs_to_shp

# Set region
tmpRst = rst_to_grs(RST_TEMPLATE, fprop(RST_TEMPLATE, 'fn'), as_cmd=True)
rst_to_region(tmpRst)

for rst in rstFiles:
    # Add data to GRASS GIS
    grsRst = rst_to_grs(rst, fprop(rst, 'fn'), as_cmd=True)
    
    # Convert to Polygon
    shpGrs = rst_to_polyg(
        grsRst, grsRst + "_shp",
        rstColumn="value", gisApi='grasscmd'
    )
    
    # Export to File
    grs_to_shp(shpGrs, os.path.join(OUT_FOLDER, grsRst + '.shp'), 'area')